In [1]:
#pip install spotipy --upgrade              # Uncomment this and run it if you haven't installed spotipy before


In [2]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import re, glob
import os, sys
from scipy import stats

import spotipy                          # Set up a config file with ckey and skey. These are available if you go
from spotipy.oauth2 import SpotifyClientCredentials      # to https://developer.spotify.com/, click on Dashboard 
                                        # from the horizontal black menu, login with your normal user info. Click 
# Import Keys                           # on "create an app" if you haven't yet, it doesn't matter what you call it.
from config import clientID, clientSEC           # Then click into
                                        # your project and you should see Client ID and Client Secret. Those are your 
                                        # ckey and skey. 

In [3]:
# Setting up Spotify API info

client_credentials_manager = SpotifyClientCredentials(client_id=clientID, client_secret=clientSEC)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



TIP: This next section assumes that you have already downloaded the csv files with the Top200 charts for the country you are working on:

1. Create a subfolder in the folder this notebook is located in called "input_files". Add the owid-covid-data.csv file there, you'll need that later. Then make another subfolder inside input_files called "spotify_top200_charts". Save the csv files you download there. 
2. Go to https://spotifycharts.com
2. Choose the country you want to work on. 
3. Download Weekly Top200 charts for 2019 and 2020, 1 chart per month. We agreed as a group to download the chart from last week of each month, to keep things consistent. Save them in the "spotify_top200_charts" folder you set up. 

In [4]:
# Create dataframe from weekly chart data


path = r"input_files/spotify_top200_charts/*.csv"                # The path requires "".csv" at end of file name. 
                                                                 # This is to prevent the program from blowing up
                                                                 # when it hits some kind of hidden file. 

country_tracks_df = pd.read_csv(glob.glob(path)[0], header=1)    # Sets up **main dataframe** with data from **FIRST** file
string = str(glob.glob(path)[0])                                 # in the folder.
year_month, = re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)   # This line extracts the year and month from the 
                                                                 # **file name**
country_tracks_df[['Year']] = year_month[0]
country_tracks_df[['Month']] = year_month[1]
country_tracks_df[['yyyy-mm']] = str(year_month[0]) + "-" + str(year_month[1]) 
Tot_Streams1 = country_tracks_df['Streams'].sum()                # Find out total streams in FIRST file in folder.
country_tracks_df[['Stream %']] = country_tracks_df['Streams'] / Tot_Streams1    # New column with % of streams

for file in glob.glob(path)[1:]:                                 # Now that you have the dataframe set up from the 
        temp_df = pd.read_csv(file, header=1)                    # first file in the folder, this iterates through 
        string = str(file)                                       # remaining files
        year_month, = re.findall(r"ly-(\d\d\d\d)-(\d\d)-\d\d", string)
        #print (year_month)
        Tot_Streams2 = temp_df['Streams'].sum()
        temp_df[['Year']] = year_month[0]
        temp_df[['Month']] = year_month[1]
        temp_df[['yyyy-mm']] = str(year_month[0]) + "-" + str(year_month[1])
        temp_df[['Stream %']] = temp_df['Streams'] / Tot_Streams2
        country_tracks_df = pd.concat([country_tracks_df,        # Adds temperary datafame to end of main dataframe
                                       temp_df])                 # as new rows.  ¿¿¿???? 
        

                                                 
country_tracks_df = country_tracks_df.sort_values(['Year','Month'])  # Sort the new dataframe by year and month




                                                                 # You should get 4,800 rows (24 months x 200 
                                                                 # tracks per month)
country_tracks_df

,Position,Track Name,Artist,Streams,URL,Year,Month,yyyy-mm,Stream %
0,1,7 rings,Ariana Grande,562985,https://open.spotify.com/track/14msK75pk3pA33p...,2019,01,2019-01,0.033085
1,2,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,389293,https://open.spotify.com/track/3KkXRkHbMCARz0a...,2019,01,2019-01,0.022878
2,3,Wow.,Post Malone,345858,https://open.spotify.com/track/6MWtB6iiXyIwun0...,2019,01,2019-01,0.020325
3,4,Without Me,Halsey,280760,https://open.spotify.com/track/5p7ujcrUXASCNwR...,2019,01,2019-01,0.016500
4,5,Sweet but Psycho,Ava Max,276126,https://open.spotify.com/track/25sgk305KZfyuqV...,2019,01,2019-01,0.016227
...,...,...,...,...,...,...,...,...,...
195,196,goosebumps,Travis Scott,51789,https://open.spotify.com/track/6gBFPUFcJLzWGx4...,2020,12,2020-12,0.002946
196,197,Thinking out Loud,Ed Sheeran,51607,https://open.spotify.com/track/1Slwb6dOYkBlWal...,2020,12,2020-12,0.002936
197,198,Santa Claus Is Coming To Town,The Jackson 5,51569,https://open.spotify.com/track/6a1hzBiTkgqTsDA...,2020,12,2020-12,0.002934
198,199,Take It Easy - 2013 Remaster,Eagles,51254,https://open.spotify.com/track/4yugZvBYaoREkJK...,2020,12,2020-12,0.002916


In [ ]:
# Get Track IDs
track_names = country_tracks_df['Track Name'].to_list()           # Set up list of tracks to iterate through         
track_ids = []                                                    # Empty list to record track IDs into

for track in track_names:                                         # Heads up: with 4800 tracks to process, this takes
    song_results = sp.search(q=track, type='track', limit=1)      # awhile 
    try:
        track_ids.append(song_results['tracks']['items'][0]['id']) # Prevents program from blowing up - few tracks
        print (f"{track} song ID : {song_results['tracks']['items'][0]['id']}")   # Just to let you know it's working
    except IndexError:                                             # lack track ids
        track_ids.append(np.nan)                                   # nan if nothing

7 rings song ID : 6ocbgoVGwYJhOv1GgI9NsF
Sunflower - Spider-Man: Into the Spider-Verse song ID : 3KkXRkHbMCARz0aVfEt68P
Wow. song ID : 7xQAfvXzm3AkraOtGPWIZg
Without Me song ID : 7lQ8MOhq6IN2w8EYcFNSUk
Sweet but Psycho song ID : 7DnAm9FOTWE3cUvso43HhI
Dancing With A Stranger (with Normani) song ID : 6Qs4SXO9dwPj5GKvVOv8Ki
MIDDLE CHILD song ID : 2JvzF1RMd7lE3KmFlsyZD8
thank u, next song ID : 3e9HZxeyfWwjeyPAMmWSSQ
Shallow song ID : 2VxeLyX666F8uXCJ0dZF8B
Better song ID : 6gdLyYNuhWBzqwlOFSsXC7
Eastside (with Halsey & Khalid) song ID : 0d2iYfpKoM0QCKvcLCkBao
Shotgun song ID : 4ofwffwvvnbSkrMSCKQDaC
Happier song ID : 2dpaYNEQHiRxtZbfNsse99
Saturday Nights song ID : 5kfNriitmkNE8mUbZ7gbq8
Body (feat. brando) song ID : 0BrDfbNc3HiIHPGfTT8DRc
Giant (with Rag'n'Bone Man) song ID : 5itOtNx0WxtJmi1TQ3RuRd
SICKO MODE song ID : 2xLMifQCjDGFmkHkpNLD9h
Soaked song ID : 4zIiarkbaDt2cm6sukb1Xt
Nothing Breaks Like a Heart (feat. Miley Cyrus) song ID : 27rdGxbavYJeBphck5MZAF
Close To Me (with Diplo) (f

I'm Yours song ID : 1EzrEOXmMH3G43AXT1y7pA
Suncity (feat. Empress Of) song ID : 6SkHm6poyKfzV6kc9any8k
Tennessee Whiskey song ID : 3fqwjXwUGN6vbzIwvyFMhx
Splashin song ID : 3KLo7ByvjCNyBrfz5DZ2mO
Brown Eyed Girl song ID : 3yrSvpt2l1xhsV9Em88Pul
Happy Now song ID : 4keoy2fqgwGnbWlm3ZVZFa
Born To Be Yours song ID : 0WVAQaxrT0wsGEG4BCVSn2
Castle on the Hill song ID : 6PCUP3dWmTjcTtXY02oFdT
1950 song ID : 0CZ8lquoTX2Dkg7Ak2inwA
Summer Of '69 song ID : 0GONea6G2XdnHWjNZd6zt3
Let Me Go (with Alesso, Florida Georgia Line & watt) song ID : 5Gu0PDLN4YJeW75PpBSg9p
No Stylist (feat. Drake) song ID : 04MLEeAMuV9IlHEsD8vF6A
IDGAF song ID : 76cy1WJvNGJTj78UqeA5zr
Trampoline song ID : 0lsRatBUs9HNIZAmoGABzk
Rocket Man (I Think It's Going To Be A Long, Long Time) song ID : 3gdewACMIVMEWVbyb8O9sY
First Off (feat. Travis Scott) song ID : 3WRIaWsws011vHMd9uzPjG
Party In The U.S.A. song ID : 5Q0Nhxo0l2bP3pNjpGJwV1
REEL IT IN song ID : 5qHirGR7M9tdm6C17DlzSY
I Found You (with Calvin Harris) song ID : 5sdb5

I Like It song ID : 58q2HKrzhC3ozto2nDdN4z
1999 song ID : 2QSUyofqpGDCo026OPiTBQ
Hotel California - 2013 Remaster song ID : 40riOy7x9W7GXjyGp4pjAv
Splashin song ID : 3KLo7ByvjCNyBrfz5DZ2mO
Under Pressure - Remastered 2011 song ID : 11IzgLRXV7Cgek3tEgGgjw
The Middle song ID : 6GG73Jik4jUlQCkKg9JuGO
Meant to Be (feat. Florida Georgia Line) song ID : 7iDa6hUg2VgEL1o1HjmfBn
Peer Pressure (feat. Julia Michaels) song ID : 2hFFke3WXVA9mV31XNy0dP
Falling Down - Bonus Track song ID : 4jvjzW7Hm0yK4LvvE0Paz9
Mr. Brightside song ID : 7oK9VyNzrYvRFo7nQEYkWN
Woke Up Late song ID : 4oPWkP9zQ5ANBbfdHGOvKF
Young Dumb & Broke song ID : 5Z3GHaZ6ec9bsiI5BenrbY
Armed And Dangerous song ID : 5wujBwqG7INdStqGd4tRMX
What A Time (feat. Niall Horan) song ID : 5WtfUKzXircvW8l5aaVZWT
Up There song ID : 2rKmNEYrQxaOPZrOWKZpOc
These Days (feat. Jess Glynne, Macklemore & Dan Caplen) song ID : 5CLGzJsGqhCEECcpnFQA8x
Love Lies (with Normani) song ID : 45Egmo7icyopuzJN0oMEdk
Perfect song ID : 0tgVpDi06FyKpA1z0VMD4v
Thu

bellyache song ID : 51NFxnQvaosfDDutk0tams
Jocelyn Flores song ID : 7m9OqQk4RVRkw9JJdeAw96
Africa song ID : 2374M0fQpWi3dLnB54qaLX
Moonlight song ID : 0JP9xo3adEtGSdUEISiszL
Psycho (feat. Ty Dolla $ign) song ID : 3swc6WTsr7rl9DqQKQA55C
Drip Too Hard (Lil Baby & Gunna) song ID : 78QR3Wp35dqAhFEc2qAGjE
Look Back at It song ID : 3Ol2xnObFdKV9pmRD2t9x8
6 Figures song ID : 1Q8OgsmxcnoGev6T7KUenD
Last Hurrah song ID : 0i0wnv9UoFdZ5MfuFGQzMy
So Close song ID : 7FjZU7XFs7P9jHI9Z0yRhK
365 song ID : 2XWjPtKdi5sucFYtVav07d
Undecided song ID : 58oryymPZNdtS2z8Une6NJ
Don’t Give It Up song ID : 0gTJK4pCZMh8PxYdaAgf2B
Let You Love Me song ID : 5Fla3zyOCcIRqrDoKLU0DP
Dreams - 2004 Remaster song ID : 0ofHAoxe9vBkTCp2UQIavz
Don’t Forget Your Roots song ID : 47xUzcYTwMWy9O273o2JeU
Shape of You song ID : 7qiZfU4dY1lWllzX7mPBI3
Wake Up in the Sky song ID : 0QIjsbm2fh1cJ45XO9eGqq
Drift Away song ID : 0RFgvrhkf9FiDRLA0BhzpZ
Ruin My Life song ID : 5tAa8Uaqr4VvA3693mbIhU
Rise song ID : 69Sy7207dnixZ6w7RSV9Kb
S

Giant (with Rag'n'Bone Man) song ID : 5itOtNx0WxtJmi1TQ3RuRd
I Don't Belong In This Club song ID : 6N22Yl2nHPnksP1hu3mxJE
thank u, next song ID : 3e9HZxeyfWwjeyPAMmWSSQ
Please Me song ID : 0PG9fbaaHFHfre2gUVo7AN
Bohemian Rhapsody - Remastered 2011 song ID : 7tFiyTwD0nx5a1eklYtX2J
i love you song ID : 6CcJMwBtXByIz4zQLzFkKc
Body (feat. brando) song ID : 0BrDfbNc3HiIHPGfTT8DRc
MONOPOLY (with Victoria Monét) song ID : 6tsOcBnaKgzK22yEiqRh8P
Youngblood song ID : 2iUXsYOEPhVqEBwsqP70rE
Paradise song ID : 0Rx0DJI556Ix5gBny6EWmn
Earth song ID : 2S1LebN6AXXQqJolBxlWgO
Be Alright song ID : 3EPXxR3ImUwfayaurPi3cm
ilomilo song ID : 7qEKqBCD2vE5vIBsrUitpD
Going Bad (feat. Drake) song ID : 2IRZnDFmlqMuOrYOLnZZyc
Always Remember Us This Way song ID : 2rbDhOo9Fh61Bbu23T2qCk
SICKO MODE song ID : 2xLMifQCjDGFmkHkpNLD9h
Better Now song ID : 7dt6x5M1jzdTEt8oCbisTK
Bright Blue Skies song ID : 5zxwumO0laRosoeOvbVSvc
ocean eyes song ID : 7hDVYcQq6MxkdJGweuCtl9
Pure Water (with Migos) song ID : 3j84U36KvLeXN

...

Stuck with U (with Justin Bieber) song ID : 4HBZA5flZLE435QTztThqH
Blinding Lights song ID : 0VjIjW4GlUZAMYd2vXMi3b
In the Air song ID : 18AXbzPzBS8Y3AkgSxzJPb
death bed (coffee for your head) song ID : 7eJMfftS33KTjuF7lTsMCx
Supalonely song ID : 4nK5YrxbMGZstTLbvj6Gxw
THE SCOTTS song ID : 39Yp9wwQiSRIDOvrVg7mbk
Toosie Slide song ID : 127QTOFJsJQp5LbJbu3A1y
Go Crazy song ID : 1IIKrJVP1C9N7iPtG6eOsK
Blueberry Faygo song ID : 6wJYhPfqk3KGhHRG76WzOh
Intentions (feat. Quavo) song ID : 4umIPjkehX1r7uhmGvXiSV
Don't Start Now song ID : 6WrI0LAC5M1Rw2MnX2ZvEg
Savage Remix (feat. Beyoncé) song ID : 5v4GgrXPMghOnBBLmveLac
Sunday Best song ID : 1Cv1YLb4q0RzL6pybtaMLo
GOOBA song ID : 4NhDYoQTYCdWHTvlbGVgwo
The Box song ID : 0nbXyq5TXYPCO7pr3N8S4I
Watermelon Sugar song ID : 6UelLqGlWMcVH1E5c4H7lY
ily (i love you baby) (feat. Emilee) song ID : 62aP9fBQKYKxi7PDXwcUAS
If the World Was Ending - feat. Julia Michaels song ID : 2kJwzbxV2ppxnQoYw4GLBZ
Before You Go song ID : 2gMXnyrvIjhVBUZwvLZDMP
Break My

Sweet Home Alabama song ID : 7e89621JPkKaeDSTQ3avtg
Redbone song ID : 0wXuerDYiBnERgIpbb3JBR
Will - Remix song ID : 6WI33r2QRxyxttn3KH0XkC
No Idea song ID : 7AzlLxHn24DxjgQX73F9fU
Road Trip song ID : 7Gpr3kKk4BMgItz6UbI73q
I Like Me Better song ID : 2P91MQbaiQOfbiz9VqhqKQ
Tennessee Whiskey song ID : 3fqwjXwUGN6vbzIwvyFMhx
OUT WEST (feat. Young Thug) song ID : 6gi6y1xwmVszDWkUqab1qw
Closer song ID : 7BKLCZ1jbUBVqRi2FVlTVw
Truth Hurts song ID : 3HWzoMvoF3TQfYg4UPszDq
Riptide song ID : 3JvrhDOgAt6p7K8mDyZwRd
My Oh My (feat. DaBaby) song ID : 3yOlyBJuViE2YSGn3nVE1K
Happier song ID : 2dpaYNEQHiRxtZbfNsse99
Wow. song ID : 7xQAfvXzm3AkraOtGPWIZg
Talk (feat. Disclosure) song ID : 6TqXcAFInzjp0bODyvrWEq
Angels & Demons song ID : 4GUj9MfPPkjpZskxb45uiX
Rocketship song ID : 1IqMA5kXJmSHQFp1pmwONM
Summer Of '69 song ID : 0GONea6G2XdnHWjNZd6zt3
Drift Away song ID : 0RFgvrhkf9FiDRLA0BhzpZ
South of the Border (feat. Camila Cabello & Cardi B) song ID : 4vUmTMuQqjdnvlZmAH61Qk
Myron song ID : 56uXDJRCuo

Perfect song ID : 0tgVpDi06FyKpA1z0VMD4v
Lost song ID : 3GZD6HmiNUhxXYf8Gch723
rockstar (feat. 21 Savage) song ID : 0e7ipj03S05BNilyu5bRzt
Brown Eyed Girl song ID : 3yrSvpt2l1xhsV9Em88Pul
Wait a Minute! song ID : 0y60itmpH0aPKsFiGxmtnh
Yummy song ID : 41L3O37CECZt3N7ziG2z7l
Bohemian Rhapsody - Remastered 2011 song ID : 7tFiyTwD0nx5a1eklYtX2J
The Less I Know The Better song ID : 6K4t31amVTZDgR3sKmwUJJ
IDK You Yet song ID : 0j1Ia2lQWrcXrQZI4AdJlk
Higher Love song ID : 6oJ6le65B3SEqPwMRNXWjY
Hotel California - 2013 Remaster song ID : 40riOy7x9W7GXjyGp4pjAv
Trampoline (with ZAYN) song ID : 1iQDltZqI7BXnHrFy4Qo1k
Best Part (feat. Daniel Caesar) song ID : 4OBZT9EnhYIV17t4pGw7ig
Shotgun song ID : 4ofwffwvvnbSkrMSCKQDaC
Find an Island song ID : 4Ve0Jx7MXjU4aPrFHJRZK7
The Chain - 2004 Remaster song ID : 5e9TFTbltYBg2xThimr0rU
Don’t Give It Up song ID : 0gTJK4pCZMh8PxYdaAgf2B
MIDDLE CHILD song ID : 2JvzF1RMd7lE3KmFlsyZD8
Redbone song ID : 0wXuerDYiBnERgIpbb3JBR
SAD! song ID : 3ee8Jmje8o58CHK66Qr

HIGHEST IN THE ROOM song ID : 3eekarcy7kvN4yt5ZFzltW
Robbery song ID : 6Hj9jySrnFppAI0sEMCZpJ
Mood Swings (feat. Lil Tjay) song ID : 5rZlwNFl01HqLWBQGryKSm
WRONG (feat. Lil Mosey) song ID : 1EWkw4Fa6IlnsAihLUlFFM
Pump It Up song ID : 4tcPIwy0UvLYjhXLrMyx89
July song ID : 6J2LdBN97cDWn0MLxYh9HB
Godzilla (feat. Juice WRLD) song ID : 7FIWs0pqAYbP91WWM0vlTQ
Ballin' (with Roddy Ricch) song ID : 3QzAOrNlsabgbMwlZt7TAY
Life Is Good (feat. Drake) song ID : 5yY9lUy8nbvjM1Uyo1Uqoc
bad guy song ID : 2Fxmhks0bxGSBdJ92vM42m
GOOBA song ID : 4NhDYoQTYCdWHTvlbGVgwo
Don’t Forget Your Roots song ID : 47xUzcYTwMWy9O273o2JeU
Go Your Own Way - 2004 Remaster song ID : 4xh7W7tlNMIczFhupCPniY
you broke me first song ID : 45bE4HXI0AwGZXfZtMp8JR
Falling song ID : 1ZMiCix7XSAbfAJlEZWMCp
No Guidance (feat. Drake) song ID : 6XHVuErjQ4XNm6nDPVCxVX
Señorita song ID : 0TK2YIli7K1leLovkQiNik
Raining song ID : 01Mpj13vURSO3cCLprPt5T
Mr. Brightside song ID : 7oK9VyNzrYvRFo7nQEYkWN
Jocelyn Flores song ID : 7m9OqQk4RVRkw9

Still Don't Know My Name song ID : 6N22FZs2ZhPBYi3b9XPajV
Dreams - 2004 Remaster song ID : 0ofHAoxe9vBkTCp2UQIavz
Shallow song ID : 2VxeLyX666F8uXCJ0dZF8B
Memories song ID : 2b8fOow8UzyDFAE27YhOZM
Dior song ID : 79s5XnCN4TJKTVMSmOx8Ep
POPSTAR (feat. Drake) song ID : 6EDO9iiTtwNv6waLwa1UUq
Chicago Freestyle (feat. Giveon) song ID : 4wVOKKEHUJxHCFFNUWDn0B
Tap In song ID : 4Hpib09wXgD84w4uwTPnYD
The Box song ID : 0nbXyq5TXYPCO7pr3N8S4I
Toosie Slide song ID : 127QTOFJsJQp5LbJbu3A1y
Lucid Dreams song ID : 285pBltuF7vW8TeWk8hdRR
Sunflower - Spider-Man: Into the Spider-Verse song ID : 3KkXRkHbMCARz0aVfEt68P
Robbery song ID : 6Hj9jySrnFppAI0sEMCZpJ
GREECE (feat. Drake) song ID : 35RJhm1pEovTBwnNR0zWad
July song ID : 6J2LdBN97cDWn0MLxYh9HB
Bruises song ID : 4Of7rzpRpV1mWRbhp5rAqG
Nobody's Love song ID : 3kwgqoBqTwoAH4nT29TYrq
goosebumps song ID : 6gBFPUFcJLzWGx4lenP6h2
Please Don't Go song ID : 6jg5SRvdGxvJ0DzNV0UqEK
exile (feat. Bon Iver) song ID : 4pvb0WLRcMtbPGmtejJJ6y
GO (feat. Juice WRLD) 

TIP: for this next section, add an "output_files" subfolder to export into. 

In [ ]:
# Add Track IDs to dataframe

country_tracks_df['Track ID'] = track_ids                  # Add new column with track IDs

In [ ]:
# Drop empty songs and export dataframe to csv to back it up

clean_country_tracks_df = country_tracks_df.dropna(how='any')           # Use .dropna() to remove rows with missing data

clean_country_tracks_df.to_csv("output_files/1_tracks_with_track_ids.csv", index = False)  # Back up to .csv

In [ ]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_track_ids_df = pd.read_csv("output_files/1_tracks_with_track_ids.csv")

country_track_ids_df

In [ ]:
# Use API again to get audio features

danceability = []                                     # Set up empty lists to store data in
energy = []
valence = []
tempo = []

for track in country_track_ids_df['Track ID']:           # Heads up: this takes a long time
    
    try:
        feat_results = sp.audio_features([track])
        danceability.append(feat_results[0]['danceability'])
        energy.append(feat_results[0]['energy'])
        valence.append(feat_results[0]['valence'])
        tempo.append(feat_results[0]['tempo'])
        print (f"{track} Valence Score: {feat_results[0]['valence']}")  # Just to let you see it working                                 
    
    except TypeError:                                 # Covers you in case there is missing data
        danceability.append(np.nan)
        energy.append(np.nan)
        valence.append(np.nan)
        tempo.append(np.nan)
        

In [ ]:
# Add audio features to dataframe

country_track_ids_df['Danceability'] = danceability     # Add new columns with audio features
country_track_ids_df['Valence'] = valence
country_track_ids_df['Energy'] = energy
country_track_ids_df['Tempo'] = tempo

# Add new columns with product of % and each feature
country_track_ids_df['Danceability_Stream%'] = danceability * country_track_ids_df['Stream %']    
country_track_ids_df['Valence_Stream%'] = energy * country_track_ids_df['Stream %'] 
country_track_ids_df['Energy_Stream%'] = valence * country_track_ids_df['Stream %'] 
country_track_ids_df['Tempo_Stream%'] = tempo * country_track_ids_df['Stream %'] 

In [ ]:
# Back up dataframe again to .csv
clean_country_track_ids_df = country_track_ids_df.dropna(how='any')          # Use .dropna() to remove rows with missing data

clean_country_track_ids_df.to_csv("output_files/2_tracks_with_audio_features.csv", index=False) #Back up the dataframe to csv again

clean_country_track_ids_df

In [ ]:
# Continue from the backup csv file in case there is some kind of interruption to the notebook and you lose the 
# data from the API calls.

country_tracks_data_df = pd.read_csv("output_files/2_tracks_with_audio_features.csv")

country_tracks_data_df.head()

In [ ]:
# Use groupby to get average valence of the 200 songs in each month

country_tracks_data_groupby = country_tracks_data_df.groupby(["Year", 'Month'], as_index=False)['Valence_Stream%'].sum()

country_tracks_data_groupby

In [ ]:
# Set up some basic plt formatting configurations

plt.rc('font', size=12)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=20)

In [ ]:
# Plot a comparison of 2019 2020 valence scores

# Set up lists to plot

valence_2019 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2019]

valence_2020 = country_tracks_data_groupby[country_tracks_data_groupby['Year'] == 2020]
valence_2020.drop('Year', inplace=True, axis=1)

fig= plt.figure(figsize=(12,8))                                      # Set up figure size
fig.suptitle('SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH')         # Set up main title

y_axis = valence_2019['Valence_Stream%']
x_axis = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',                  # Set up x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

plt.plot(x_axis, valence_2019['Valence_Stream%'], label="2019 Weighted Avg Valence", marker='o', color='darkred')  # Plot 2019
plt.plot(x_axis, valence_2020['Valence_Stream%'], label="2020 Weighted Avg Valence", marker='o', color='steelblue')  # Plot 2020

plt.xlabel('Months')                                                 # Set up axis titles
plt.ylabel('Valence Score')

plt.xlim(-0.75, len(x_axis)-0.25)                                    # Set up axis limits
plt.ylim(0, 1)

plt.legend()                                                         # Include the legend

plt.show()


In [ ]:
# Compare valence scores with covide infection rate

covid_df = pd.read_csv("input_files/owid-covid-data.csv")                # read the covid data file

country_covid_df = covid_df.loc[covid_df['location'] == 'New Zealand']   # Filter for country of your choice

country_covid_df.head()


In [ ]:
# Filter data for 2020, and add a 'month' column

country_covid__2020_df = country_covid_df[country_covid_df.date.str.contains(r'2020.*')]

country_covid__2020_df['Month'] = ''
country_covid__2020_df.head()

In [ ]:
# Extract the month from the 'date' column and add it to the new 'month' column, for sorting later

for index, row in country_covid__2020_df.iterrows():
    month, = re.findall(f"2020-(\d\d)-", row['date'])
    country_covid__2020_df.at[index, 'Month'] = int(month)  

country_covid__2020_df.head()

In [ ]:
# Create a groupby to get the sum of new cases in each month

country_covid__2020_groupby = country_covid__2020_df.groupby(['Month'], as_index=False)['new_cases'].sum()
country_covid__2020_groupby


In [ ]:
# TIP: This next section is to add missing months. In the case of New Zealand, there was no data for January
# For other countries, it might vary. Here's how I added January to the dataframe:

country_covid__2020_groupby.loc[-1] = [1, 0]             # This adds a 1 (month of January) in the first columne 
                                                         # index 0), in the last row of the dataframe. 

country_covid__2020_groupby.index = country_covid__2020_groupby.index + 1   # shifts the index
country_covid__2020_groupby = country_covid__2020_groupby.sort_index()      # sorts by index

country_covid__2020_groupby = country_covid__2020_groupby.rename(columns={"new_cases": "New Cases"})

country_covid__2020_groupby


In [ ]:
# Merge the dataframes into one nice comparison dataframe to scatter plot

country_covid_valence_df = pd.merge(valence_2020, country_covid__2020_groupby, on="Month")

country_covid_valence_df

In [ ]:
# Add a new cases per million column

country_polulation = 5002100  #TIP: This the population of New Zealand. Adjust for your country

country_covid_valence_df['New Cases Per Million'] = country_covid_valence_df['New Cases'] / country_polulation *1000000

country_covid_valence_df

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

fig, ax1 = plt.subplots(figsize=(12,8))                   # Set up subplot figure and size
fig.suptitle('NEW ZEALAND: SPOTIFY LISTENER VALENCE PREFERENCE BY MONTH COMPARED TO NEW COVID CASES')

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',       # Set up shared x axis
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

ax1.set_xlabel('Months')                

ax1.set_ylabel('Average Valence Scores')                  # Set up first plot 
ax1.set_ylim([.2, .8])

ax1_ydata = country_covid_valence_df['Valence_Stream%']
ax1.plot(months, ax1_ydata, label="Weighted Avg Valence Scores", marker='o', color='darkred')                       
                        
ax2 = ax1.twinx()                                         # Set up second plot 
ax2.set_ylabel('New Cases Per Month')
ax2_ydata = country_covid_valence_df['New Cases Per Million']
ax2.set_ylim([0, ax2_ydata.max()+20])
ax2.plot(months, ax2_ydata, label="New Covid Cases Per Million", marker='o', color='steelblue')


lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

plt.show()


In [ ]:
# This is a mini function that adds a qualitative label to the correlation r score in the scatter plot

def r_label(r):
    abs_r = abs(r)
    if abs_r >= .8 : return "Very Strong"
    elif abs_r >= .6 : return "Strong"
    elif abs_r >= .4: return "Moderate"
    elif abs_r >= .2: return "Low"
    else: return "Negligible"

In [ ]:
# Line plot relationship between Valence scores and New Cases per Million with shared x axis and duel y axes

x_axis = country_covid_valence_df['Valence_Stream%']                                 # Set up axes
y_axis = country_covid_valence_df['New Cases Per Million']

slope, intercept, rvalue, pvalue, stderr = stats.linregress(x_axis, y_axis)  # Get elements of regression equation
regress_values = x_axis * slope + intercept                                  # Calculate regression values

plt.figure(figsize=(12, 8))
plt.title('NEW ZEALAND: SPOTIFY LISTENER VALENCE PREFERENCE VS. NEW COVID CASES') # CHANGE TITLE TO REFLECT YOUR COUNTRY

plt.xlabel(f"New Covid Cases")                                               # Set x axis label for subplot                  
plt.ylabel(f"Valence")                                                       # Set title for subplot

r = round(stats.pearsonr(x_axis, y_axis)[0],2)                       # Calculate correlation coefficient
rlabel = r_label(r)                                                 # Call function to create a label for the r number

line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))    # Ression formula

plt.scatter(x_axis, y_axis, facecolors="darkred",  alpha=.5,        # Plot the scatter chart
label=f"r = {r}\nCorrelation is {rlabel}\n{line_eq}" ) 

plt.plot(x_axis, regress_values, color="steelblue")                 # Plot the regression line    

plt.legend()                                                        # Add the legend
    
plt.savefig("output_files/valence_vs_newcases.png")                 # Save the png file   

plt.show()